In [5]:
### Function Imports
from AdvPairsFunctions import import_fx_data, plot_forex, daily_returns, get_beta, beta_backtest, performance_metrics


In [6]:
### Call Function to Load in Rate Data
all_forex_pairs = [
        'EURUSD=X', 'GBPUSD=X', 'USDJPY=X', 'USDCHF=X', 'AUDUSD=X', 'NZDUSD=X',
        'USDCAD=X', 'EURGBP=X', 'EURJPY=X', 'EURCHF=X', 'EURAUD=X', 'EURNZD=X',
        'EURCAD=X', 'GBPAUD=X', 'GBPJPY=X', 'GBPCHF=X', 'AUDJPY=X', 'AUDCHF=X',
        'AUDNZD=X', 'NZDJPY=X', 'NZDCHF=X', 'CADJPY=X', 'CADCHF=X', 'CHFJPY=X'
    ]
start_date = '2019-01-01'
forex_data = import_fx_data(all_forex_pairs, start_date)
forex_data.tail()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,Date,EURUSD=X,GBPUSD=X,USDJPY=X,USDCHF=X,AUDUSD=X,NZDUSD=X,USDCAD=X,EURGBP=X,EURJPY=X,...,GBPJPY=X,GBPCHF=X,AUDJPY=X,AUDCHF=X,AUDNZD=X,NZDJPY=X,NZDCHF=X,CADJPY=X,CADCHF=X,CHFJPY=X
1424,2024-06-18,1.074068,1.271600,157.699997,0.88830,0.662208,0.613800,1.37133,0.84480,169.369003,...,200.488998,1.12942,104.374001,0.58800,1.078544,96.755997,0.54460,114.992996,0.64774,177.498001
1425,2024-06-19,1.074229,1.270745,157.871002,0.88418,0.666120,0.614058,1.37162,0.84500,169.494003,...,200.561996,1.12360,105.126999,0.58895,1.084624,96.927002,0.54283,115.059998,0.64458,178.477005
1426,2024-06-20,1.075153,1.272200,157.957993,0.88430,0.667530,0.613941,1.37058,0.84490,169.763000,...,200.927002,1.12498,105.432999,0.59031,1.087239,96.960999,0.54286,115.244003,0.64522,178.587997
1427,2024-06-21,1.070641,1.266047,158.931000,0.89149,0.666070,0.612280,1.36847,0.84561,170.151001,...,201.205994,1.12856,105.857002,0.59375,1.087890,97.295998,0.54574,116.140999,0.65140,178.261002
1428,2024-06-24,1.068719,1.263392,159.690994,0.89385,0.662941,0.611023,1.37068,0.84566,NaN,...,201.753006,1.12928,105.849998,NaN,NaN,NaN,0.54573,116.513000,0.65206,178.651001


In [7]:
### Call Function to Calculate Daily Returns and Build Results df
daily_rets = daily_returns(forex_data)
print(daily_rets.head())


        Date  EURUSD=X  GBPUSD=X  USDJPY=X  USDCHF=X  AUDUSD=X  NZDUSD=X  \
0 2019-01-02 -0.002728  0.001275  0.000347 -0.001535  0.000874 -0.001685   
1 2019-01-03 -0.012529 -0.018219 -0.020307  0.006823 -0.018294 -0.011921   
2 2019-01-04  0.006447  0.008537  0.003416 -0.001618  0.012611  0.008280   
3 2019-01-07  0.001700  0.008405  0.006623 -0.001358  0.016777  0.008220   
4 2019-01-08  0.006073  0.004015  0.000866 -0.006756  0.003108  0.001513   

   USDCAD=X  EURGBP=X  EURJPY=X  ...  GBPJPY=X  GBPCHF=X  AUDJPY=X  AUDCHF=X  \
0  0.017254 -0.004211 -0.002310  ...  0.001848 -0.000287  0.001425 -0.001010   
1  0.000426  0.006177 -0.032525  ... -0.038425 -0.011802 -0.038300 -0.011632   
2 -0.011180 -0.002367  0.009811  ...  0.012321  0.007289  0.016172  0.011184   
3 -0.008124 -0.006630  0.008364  ...  0.014999  0.006907  0.023502  0.015210   
4 -0.005483  0.002143  0.006776  ...  0.004870 -0.002780  0.003467 -0.003817   

   AUDNZD=X  NZDJPY=X  NZDCHF=X  CADJPY=X  CADCHF=X  CHFJPY=X 

In [9]:
### Function Call to Compute and Store All Results
'''
Beta is a measure of the sensitivity or volatility of an asset's returns in relation 
to the returns of a benchmark or market index. In the context of foreign exchange (FX) 
trading, beta represents how much the returns of one currency pair move in relation to 
the returns of another currency pair.
Beta = 1: The returns of the currency pair move in tandem with the benchmark. If the 
          benchmark's return increases by 1%, the currency pair's return also increases 
          by 1%, on average.

Beta > 1: The currency pair is more volatile than the benchmark. For example, if beta 
          is 1.5, then the currency pair's return is expected to increase by 1.5% for 
          every 1% increase in the benchmark's return, on average.

Beta < 1: The currency pair is less volatile than the benchmark. For example, if beta is 
          0.5, then the currency pair's return is expected to increase by 0.5% for every 
          1% increase in the benchmark's return, on average.

Beta = 0: There is no relationship between the currency pair's returns and the benchmark's returns.

Negative Beta: The currency pair moves in the opposite direction of the benchmark. For 
               example, if beta is -1, then the currency pair's return is expected to decrease
               by 1% for every 1% increase in the benchmark's return, on average.
'''
# Define base pairs and benchmark pairs and long position
long_pos = 1000
base_pair = 'USDJPY=X'
benchmark_pair = 'EURUSD=X'

beta = get_beta(daily_rets, base_pair, benchmark_pair)
short_pos, hedged_cumulative_rets, unhedged_cumulative_rets = beta_backtest(long_pos, base_pair, benchmark_pair, daily_rets, beta)
metrics = performance_metrics(long_pos, hedged_cumulative_rets, unhedged_cumulative_rets, daily_rets)



Metric                                  Hedged        Unhedged
Total Returns                         419.0037        456.6359
Annualized Returns                      1.9035          1.9478
Mean Return                          1100.3879       1115.9196
Standard Deviation                    137.8943        155.2641
Sharpe Ratio                          126.6776        114.0939

The Sharpe ratio measures the risk-adjusted return of an investment.
In this instance, the hedged strategy has a Sharpe ratio of 126.6776,
which means for every unit of risk, the hedged portfolio is generating 126.6776 units of return above the risk-free rate.
The unhedged strategy has a Sharpe ratio of 114.0939,
indicating that for every unit of risk, the unhedged portfolio is generating 114.0939 units of return above the risk-free rate.
A higher Sharpe ratio typically indicates a more favorable risk-adjusted return.
